In [56]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from sklearn.metrics import mutual_info_score
from statsmodels.stats.outliers_influence import variance_inflation_factor   
from joblib import Parallel, delayed

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [95]:
df = pd.read_csv('data.csv', sep=";")
df.shape

(4424, 37)

## Target

We converted 3-class problem into 2-class problem by merging ```Enrolled``` and ```Graduate``` as one class.

The ```Target``` column gives whether the student will dropout or not where 1 means Dropout and 0 means not

In [58]:
df.Target.replace(['Enrolled', 'Graduate'], 0, inplace=True)
df.Target.replace('Dropout', 1, inplace=True)

## Nacionality

- 1 - Portuguese
- 2 - German
- 6 - Spanish
- 11 - Italian
- 13 - Dutch
- 14 - English
- 17 - Lithuanian
- 21 - Angolan
- 22 - Cape Verdean
- 24 - Guinean
- 25 - Mozambican
- 26 - Santomean
- 32 - Turkish
- 41 - Brazilian
- 62 - Romanian
- 100 - Moldova (Republic of)
- 101 - Mexican
- 103 - Ukrainian
- 105 - Russian
- 108 - Cuban
- 109 - Colombian	

### Combinations:
- 1 as "Portuguese"
- remaining as "Others"

In [59]:
df.Nacionality.replace(1, 0, inplace=True)
df.Nacionality.replace(list(range(2, 110)), 1, inplace=True)

df.Nacionality.value_counts()

0    4314
1     110
Name: Nacionality, dtype: int64

In [60]:
df[['International', 'Nacionality']].corr()

,International,Nacionality
International,1.0,1.0
Nacionality,1.0,1.0


Since there is direct relation between ```International``` and ```Nacionality```, we are dropping ```Internatinoal``` column from our dataset

In [61]:
df.drop('International', axis=1, inplace=True)

We have an unsaid rule that the qualification is directly related to the occupation of the person. Therefore, we are removing the ```Father's Qualification``` and ```Mother's Occupation``` from our dataset.

In [62]:
df.drop(['Father_qualification', 'Mother_occupation'], axis=1, inplace=True)

## Marital Status:
- 1 – single 2 – married 3 – widower 4 – divorced 5 – facto union 6 – legally separated
- Since the column had 5 status, we converted them into 2 status ```Married``` and ```Un-Married```
- Our binning is done in the following manner:
    - Married and Facto Union as ```Married``` and is labelled as ```1```
    - Remaining as ```Un-Married``` and is labelled as ```0```

In [63]:
df.Marital_status.replace([1, 3, 4, 6], 0, inplace=True)
df.Marital_status.replace([2, 5], 1, inplace=True)

df.Marital_status.value_counts()

0    4020
1     404
Name: Marital_status, dtype: int64

## Application Mode:
- 1 - 1st phase - general contingent 
- 2 - Ordinance No. 612/93 
- 5 - 1st phase - special contingent (Azores Island) 
- 7 - Holders of other higher courses 
- 10 - Ordinance No. 854-B/99 
- 15 - International student (bachelor) 
- 16 - 1st phase - special contingent (Madeira Island) 
- 17 - 2nd phase - general contingent 
- 18 - 3rd phase - general contingent 
- 26 - Ordinance No. 533-A/99, item b2) (Different Plan) 
- 27 - Ordinance No. 533-A/99, item b3 (Other Institution) 
- 39 - Over 23 years old 
- 42 - Transfer 
- 43 - Change of course 
- 44 - Technological specialization diploma holders 
- 51 - Change of institution/course 
- 53 - Short cycle diploma holders 
- 57 - Change of institution/course (International)

From above entries:
- 2, 26, 27, 10 can be clubbed into one "Others"
- 1, 5, 16 cab be combined into "First_Phase"
- 43, 51, 57 can be clubbed into "Course_Change"
- 7, 44, 53 can be clubbed into "Courses_Holders"
- 17, 18 can be clubbed into "Last_Phase"
- 15, 42 can be clubbed into "Transfer"
- 39 => "OverAge"

In [64]:
df.Application_mode.replace([2, 26, 27, 10], 'Others', inplace=True)
df.Application_mode.replace([1, 5, 16], "First_Phase", inplace=True)
df.Application_mode.replace([43, 51, 57], "Course_Change", inplace=True)
df.Application_mode.replace([7, 44, 53], "Courses_Holders", inplace=True)
df.Application_mode.replace([17, 18], "Last_Phase", inplace=True)
df.Application_mode.replace([15, 42], "Transfer", inplace=True)
df.Application_mode.replace(39, "OverAge", inplace=True)

## Application Order
- Application order (between 0 - first choice; and 9 last choice)
- Combinations:
    - 0, 1 as "First_Choice"
    - 2 as "Second_Choice"
    - 3, 4 as "Second_Last_Choice"
    - 5, 6, 9 as "Last_Choice"

In [65]:
df.Application_order.value_counts()

1    3026
2     547
3     309
4     249
5     154
6     137
9       1
0       1
Name: Application_order, dtype: int64

## Course
- 33 - Biofuel Production Technologies 
- 171 - Animation and Multimedia Design 
- 8014 - Social Service (evening attendance) 
- 9003 - Agronomy 
- 9070 - Communication Design 
- 9085 - Veterinary Nursing 
- 9119 - Informatics Engineering 
- 9130 - Equinculture 
- 9147 - Management 
- 9238 - Social Service 
- 9254 - Tourism 
- 9500 - Nursing 
- 9556 - Oral Hygiene 
- 9670 - Advertising and Marketing Management 
- 9773 - Journalism and Communication 
- 9853 - Basic Education 
- 9991 - Management (evening attendance)

Combinations: 
- 9085, 9500, 9556 as "Medical"
- 9147, 9670, 9991 as "Management"
- 33, 171, 9119 as "Technology"
- 9003, 9130 as "Nature"
- 9238, 8014 as "Social_Service"
- 9070, 9773 as "Communication"
- 9254, 9853 as "Others"

We have combined the values into the above 7 courses and will be applicabe for One-Hot Encoding.

In [66]:
df.Course.replace([9085, 9500, 9556], "Medical", inplace=True)
df.Course.replace([9147, 9670, 9991], "Management", inplace=True)
df.Course.replace([33, 171, 9119], "Technology", inplace=True)
df.Course.replace([9003, 9130], "Nature", inplace=True)
df.Course.replace([9238, 8014], "Social_Service", inplace=True)
df.Course.replace([9070, 9773], "Communication", inplace=True)
df.Course.replace([9254, 9853], "Others", inplace=True)

pd.crosstab(df.Target, df.Course)

Course,Communication,Management,Medical,Nature,Others,Social_Service,Technology
Target,,,,,,,
0,405,551,948,187,263,434,215
1,152,365,241,164,181,136,182


## Previous Qualification

- 1 - Secondary education 
- 2 - Higher education - bachelor's degree 
- 3 - Higher education - degree 
- 4 - Higher education - master's 
- 5 - Higher education - doctorate 
- 6 - Frequency of higher education 
- 9 - 12th year of schooling - not completed 
- 10 - 11th year of schooling - not completed 
- 12 - Other - 11th year of schooling 
- 14 - 10th year of schooling 
- 15 - 10th year of schooling - not completed 
- 19 - Basic education 3rd cycle (9th/10th/11th year) or equiv. 
- 38 - Basic education 2nd cycle (6th/7th/8th year) or equiv. 
- 39 - Technological specialization course 
- 40 - Higher education - degree (1st cycle) 
- 42 - Professional higher technical course 
- 43 - Higher education - master (2nd cycle)

We have combined the values into 4 generic labels 

In [67]:
df.Previous_qualification.replace([35, 34], 'Others', inplace=True)
df.Previous_qualification.replace([2, 3, 4, 5, 6, 18, 22, 39, 40, 41, 42, 43, 44], 'Graduation_plus', inplace=True)
df.Previous_qualification.replace([1, 9, 10, 12, 19, 27], 'High_School_Eq', inplace=True)
df.Previous_qualification.replace([11, 14, 15, 26, 29, 30, 36, 37, 38], 'Basic_Education', inplace=True)

pd.crosstab(df.Target, df.Previous_qualification)

Previous_qualification,Basic_Education,Graduation_plus,High_School_Eq
Target,,,
0,5,281,2717
1,5,194,1222


## Mother's Qualification

- 1 - Secondary Education - 12th Year of Schooling or Eq. 
- 2 - Higher Education - Bachelor's Degree 
- 3 - Higher Education - Degree 
- 4 - Higher Education - Master's 
- 5 - Higher Education - Doctorate 
- 6 - Frequency of Higher Education 
- 9 - 12th Year of Schooling - Not Completed 
- 10 - 11th Year of Schooling - Not Completed 
- 11 - 7th Year (Old) 
- 12 - Other - 11th Year of Schooling 
- 14 - 10th Year of Schooling 
- 18 - General commerce course 
- 19 - Basic Education 3rd Cycle (9th/10th/11th Year) or Equiv. 
- 22 - Technical-professional course 
- 26 - 7th year of schooling 
- 27 - 2nd cycle of the general high school course 
- 29 - 9th Year of Schooling - Not Completed 
- 30 - 8th year of schooling 
- 34 - Unknown 
- 35 - Can't read or write 
- 36 - Can read without having a 4th year of schooling 
- 37 - Basic education 1st cycle (4th/5th year) or equiv. 
- 38 - Basic Education 2nd Cycle (6th/7th/8th Year) or Equiv. 
- 39 - Technological specialization course 
- 40 - Higher education - degree (1st cycle) 
- 41 - Specialized higher studies course 
- 42 - Professional higher technical course 
- 43 - Higher Education - Master (2nd cycle) 
- 44 - Higher Education - Doctorate (3rd cycle)

In [68]:
df.Mother_qualification.replace([35, 34], 'Others', inplace=True)
df.Mother_qualification.replace([2, 3, 4, 5, 6, 18, 22, 39, 40, 41, 42, 43, 44], 'Graduation_plus', inplace=True)
df.Mother_qualification.replace([1, 9, 10, 12, 19, 27], 'High_School_Eq', inplace=True)
df.Mother_qualification.replace([11, 14, 26, 29, 30, 36, 37, 38], 'Basic_Education', inplace=True)

pd.crosstab(df.Target, df.Mother_qualification)

Mother_qualification,Basic_Education,Graduation_plus,High_School_Eq,Others
Target,,,,
0,1056,437,1475,35
1,530,192,601,98


## Father's Occupation

In [71]:
df.Father_occupation.replace([0, 90, 99], "Others", inplace=True)
df.Father_occupation.replace([9, 151, 152, 153, 163, 175, 192, 193, 194, 195], "Unskilled", inplace=True)
df.Father_occupation.replace([3, 4, 5, 6, 7, 8, 131, 132, 134, 135, 141, 143, 144, 154, 161, 171, 172, 174, 181, 182, 183], "Skilled", inplace=True)
df.Father_occupation.replace([1, 2, 10, 101, 102, 103, 112, 114, 121, 122, 123, 124], "Highly_Skilled", inplace=True)

pd.crosstab(df.Target, df.Father_occupation)

Father_occupation,Highly_Skilled,Others,Skilled,Unskilled
Target,,,,
0,409,71,1801,722
1,205,141,748,327


In [73]:
categorical_columns = ['Father_occupation', 'Mother_qualification', 'Previous_qualification', 'Course', 'Application_mode']
categorical_columns

['Father_occupation',
 'Mother_qualification',
 'Previous_qualification',
 'Course',
 'Application_mode']

In [74]:
dummied_df = pd.get_dummies(df, columns=categorical_columns)

In [75]:
dummied_df.shape

(4424, 54)

# Calculate VIF for continouous features

In [77]:
def calculate_vif_(X, thresh=5.0):
    variables = [X.columns[i] for i in range(X.shape[1])]
    dropped=True
    while dropped:
        dropped=False
        
        vif = Parallel(n_jobs=-1,verbose=5)(delayed(variance_inflation_factor)(X[variables].values, ix) 
                                            for ix in range(len(variables)))
 
        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print ('VIF is ' + str(max(vif)) + ' ' + 'dropping ' + X[variables].columns[maxloc])
            
            variables.pop(maxloc)
            dropped=True
 
    return variables

In [78]:
df_numerical = df[['enrollment_age', 'Curricular_units_1st_sem_credited', 'Curricular_units_1st_sem_enrolled',
       'Curricular_units_1st_sem_evaluations',
       'Curricular_units_1st_sem_approved', 'Curricular_units_1st_sem_grade',
       'Curricular_units_1st_sem_without_evaluations',
       'Curricular_units_2nd_sem_credited',
       'Curricular_units_2nd_sem_enrolled',
       'Curricular_units_2nd_sem_evaluations',
       'Curricular_units_2nd_sem_approved', 'Curricular_units_2nd_sem_grade',
       'Curricular_units_2nd_sem_without_evaluations', 'Unemployment_rate',
       'Inflation_rate', 'GDP']]

rem_cols = calculate_vif_(df_numerical)

col_to_remove = df_numerical.columns[~df_numerical.columns.isin(rem_cols)]
updated_df = dummied_df.drop(columns=col_to_remove)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  16 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  13 out of  16 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  15 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  14 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of  14 | elapsed:    0.0s r

VIF is 170.1527030518737 dropping Curricular_units_1st_sem_enrolled
VIF is 36.867565118244755 dropping Curricular_units_1st_sem_approved
VIF is 27.033349563788548 dropping Curricular_units_2nd_sem_enrolled
VIF is 24.559100874617233 dropping Curricular_units_2nd_sem_grade
VIF is 16.179782871743857 dropping Curricular_units_1st_sem_evaluations
VIF is 14.475932439124119 dropping Curricular_units_1st_sem_grade
VIF is 11.361186532510192 dropping Unemployment_rate
VIF is 10.425118633978823 dropping Curricular_units_2nd_sem_credited
VIF is 6.439192750135191 dropping Curricular_units_2nd_sem_evaluations


[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0

In [80]:
dropped_df = updated_df.drop('Marital_status', axis=1)
proxy_features = []
for i in dropped_df.columns:
    mi = mutual_info_score(updated_df.Marital_status, dropped_df[i])
    if mi > 0.1:
        proxy_features.append(i)
print(proxy_features)

['enrollment_age']


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\cluster\_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\cluster\_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\cluster\_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\cluster\_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)


## Extract features from dataset infortant to the Target

In [81]:
X = updated_df.drop('Target', axis=1)
Y = updated_df.Target


model = XGBClassifier()
model.fit(X, Y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [82]:
imp_df = pd.DataFrame({'Colname': X.columns, 'importance': model.feature_importances_})
imp_df.sort_values(by='importance', ascending=False, inplace=True)

In [96]:
imp_df.shape

(44, 2)

In [88]:
cols_to_remove = imp_df[imp_df.importance == 0.0].Colname.values

In [94]:
cols_to_remove

array(['Previous_qualification_High_School_Eq',
       'Previous_qualification_Basic_Education',
       'Application_mode_Others'], dtype=object)

## Create final CSV for easy usage

In [92]:
final_df = updated_df[imp_df.Colname[~imp_df.Colname.isin(cols_to_remove)].values]
final_df['Target'] = updated_df.Target

final_df.to_csv("dataset_featured_v2.csv", index=False)

C:\Users\avina\AppData\Local\Temp\ipykernel_128728\4118257430.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Target'] = updated_df.Target


In [93]:
final_df.shape

(4424, 42)